In [4]:
!pip install evaluate rouge_score M
!pip install rouge_score

  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 77.0 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=b92bc7c4f50c416724ff643249fe4307a7b8d704cce473fcb773a70f4867efcd
  Stored in directory: /home/sttau/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [7]:
from evaluate import load
import json
import nltk

nltk.download('punkt_tab')

# Load evaluation metrics
rouge = load('rouge')
meteor = load('meteor')

# Define your directory or list of files
input_directory = 'new_results/dev'  
output_file = 'dev_evaluation_title.json'

# Initialize a list to hold results
all_results = []

# Load the gold labels once if they are in a single file
with open('../data/AmazonReview/amazon_title_generation_outputs_dev.json', 'r') as json_file:
    golds = json.load(json_file)

# Initialize a list to hold results
all_results = []

# Function to process each file
def process_file(file_path, golds):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)

    preds = []
    labels = []

    # Assuming 'data' is a list of dictionaries
    for i, entry in enumerate(data):
        # Assuming each entry has an 'output' field for raw predictions
        raw_output = entry["output"]

        tmp = raw_output.replace("\n", "").strip()
        
        try:
            # Extract the part after the first 'The title is: '
            tmp = tmp.split('The title is: ', 1)[1]

            # Handle cases with repetition or incomplete segments
            if 'The title is:' in tmp:
                tmp = tmp.split('The title is:', 1)[0].strip()
            elif 'The title' in tmp:
                tmp = tmp.split('The title', 1)[0].strip()
            elif 'The' in tmp:
                tmp = tmp.split('The', 1)[0].strip()

            preds.append(tmp)
        except:
            preds.append(tmp)

        labels.append(golds[i]["output"].replace("\n", "").strip())

    # Compute ROUGE and METEOR scores
    rouge_results = rouge.compute(predictions=preds, references=labels)
    meteor_results = meteor.compute(predictions=preds, references=labels)

    # Round results to the thousandth place
    result = {
        "rouge-1": round(rouge_results["rouge1"], 3),
        "rouge-L": round(rouge_results["rougeL"], 3),
        "meteor": round(meteor_results["meteor"], 3)
    }

    return result

[nltk_data] Downloading package punkt_tab to /home/sttau/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sttau/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sttau/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sttau/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
import os
# Iterate over files in the directory and process them
for filename in os.listdir(input_directory):
    if filename.endswith('.json'):  # Ensure only JSON files are processed
        file_path = os.path.join(input_directory, filename)
        result = process_file(file_path, golds)
        all_results.append({"filename": filename, "result": result})

# Save all results to an output file
with open(output_file, 'w') as outfile:
    json.dump(all_results, outfile, indent=4)

print(f"Evaluation results saved to {output_file}")

Evaluation results saved to dev_evaluation_title.json
